# 05b. Prompt Tuning
## Synthetic Instruction Tuner - Alternative Adaptation Method

This notebook implements **Prompt Tuning** as an alternative adaptation method for comparison:
1. Load base model (same as SFT)
2. Initialize soft prompt embeddings
3. Train only the prompt parameters
4. Collect efficiency metrics for comparison
5. Evaluate and save the model

**Comparison with LoRA**:
| Method | Trainable Params | Memory | Training Time |
|--------|-----------------|--------|---------------|
| LoRA (r=8) | ~0.1-0.5% | Medium | Medium |
| Prompt Tuning | ~0.01% | Low | Fast |

**Expected runtime**: 3-5 hours on Colab T4

## 1. Setup & Efficiency Tracking

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Project path
PROJECT_ROOT = "/content/drive/MyDrive/synthetic-instruction-tuner"

In [ ]:
# Load configuration
import json

with open(f"{PROJECT_ROOT}/config.json", 'r') as f:
    config = json.load(f)

print("Configuration loaded!")

In [ ]:
# Install libraries with latest compatible versions (avoid dependency conflicts)
!pip install -q --upgrade transformers>=4.41.0 peft>=0.7.0 trl>=0.7.4 datasets>=2.16.0 accelerate>=0.25.0 bitsandbytes>=0.41.3

print("✅ Libraries installed successfully!")

In [ ]:
import torch
import json
import os
import time
from datetime import datetime
from datasets import Dataset
import gc

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Efficiency Metrics Tracker
class EfficiencyTracker:
    """Track efficiency metrics for adaptation method comparison."""
    
    def __init__(self, method_name: str):
        self.method_name = method_name
        self.metrics = {
            "method": method_name,
            "memory_allocated_gb": [],
            "memory_reserved_gb": [],
            "training_time_seconds": 0,
            "trainable_params": 0,
            "total_params": 0,
            "trainable_ratio": 0,
            "inference_tokens_per_sec": 0,
        }
        self.start_time = None
    
    def log_memory(self):
        """Log current GPU memory usage."""
        if torch.cuda.is_available():
            allocated = torch.cuda.memory_allocated() / 1e9
            reserved = torch.cuda.memory_reserved() / 1e9
            self.metrics["memory_allocated_gb"].append(allocated)
            self.metrics["memory_reserved_gb"].append(reserved)
            return {"allocated": allocated, "reserved": reserved}
        return None
    
    def start_training(self):
        """Start timing training."""
        self.start_time = time.time()
        self.log_memory()
    
    def end_training(self):
        """End timing training."""
        if self.start_time:
            self.metrics["training_time_seconds"] = time.time() - self.start_time
        self.log_memory()
    
    def log_params(self, model):
        """Log parameter counts."""
        trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
        total = sum(p.numel() for p in model.parameters())
        self.metrics["trainable_params"] = trainable
        self.metrics["total_params"] = total
        self.metrics["trainable_ratio"] = trainable / total if total > 0 else 0
    
    def log_inference_speed(self, tokens_generated: int, time_taken: float):
        """Log inference speed."""
        self.metrics["inference_tokens_per_sec"] = tokens_generated / time_taken if time_taken > 0 else 0
    
    def get_summary(self):
        """Get summary metrics."""
        summary = {
            "method": self.method_name,
            "trainable_params": self.metrics["trainable_params"],
            "total_params": self.metrics["total_params"],
            "trainable_ratio_percent": self.metrics["trainable_ratio"] * 100,
            "peak_memory_gb": max(self.metrics["memory_allocated_gb"]) if self.metrics["memory_allocated_gb"] else 0,
            "training_time_hours": self.metrics["training_time_seconds"] / 3600,
            "inference_tokens_per_sec": self.metrics["inference_tokens_per_sec"],
        }
        return summary
    
    def save(self, path: str):
        """Save metrics to JSON."""
        with open(path, 'w') as f:
            json.dump(self.get_summary(), f, indent=2)
        print(f"Metrics saved to {path}")

# Initialize tracker
tracker = EfficiencyTracker("prompt_tuning")
print("Efficiency tracker initialized!")

## 2. Load Training Data

In [ ]:
# Load SFT training data (same as LoRA)
TRAIN_PATH = f"{config['paths']['data_filtered']}/sft_train.json"
VAL_PATH = f"{config['paths']['data_filtered']}/sft_val.json"

with open(TRAIN_PATH, 'r', encoding='utf-8') as f:
    train_data = json.load(f)

with open(VAL_PATH, 'r', encoding='utf-8') as f:
    val_data = json.load(f)

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")

In [ ]:
# Convert to HuggingFace Dataset
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

print(f"Train dataset: {train_dataset}")
print(f"Val dataset: {val_dataset}")

## 3. Load Base Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Use same base model as SFT for fair comparison
BASE_MODEL_ID = config['models']['sft_base']

print(f"Loading base model: {BASE_MODEL_ID}")

In [ ]:
# 4-bit quantization (same as LoRA for fair comparison)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print("Quantization config created")

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)

# Set pad token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer.padding_side = "right"

print(f"Tokenizer loaded. Vocab size: {tokenizer.vocab_size}")

In [ ]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False

# Log initial memory
mem = tracker.log_memory()
print(f"Model loaded!")
print(f"GPU Memory: {mem['allocated']:.2f} GB allocated, {mem['reserved']:.2f} GB reserved")

## 4. Configure Prompt Tuning

In [ ]:
from peft import PromptTuningConfig, PromptTuningInit, get_peft_model, TaskType

# Prompt Tuning configuration
# num_virtual_tokens: number of soft prompt tokens to prepend
prompt_tuning_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    num_virtual_tokens=20,  # Number of soft prompt tokens
    prompt_tuning_init=PromptTuningInit.RANDOM,  # or TEXT for initialized prompts
    tokenizer_name_or_path=BASE_MODEL_ID,
)

print("Prompt Tuning config:")
print(f"  num_virtual_tokens: {prompt_tuning_config.num_virtual_tokens}")
print(f"  init_method: {prompt_tuning_config.prompt_tuning_init}")

In [ ]:
# Apply Prompt Tuning to model
model = get_peft_model(model, prompt_tuning_config)

# Log parameters
tracker.log_params(model)

print(f"\nTrainable parameters: {tracker.metrics['trainable_params']:,}")
print(f"Total parameters: {tracker.metrics['total_params']:,}")
print(f"Trainable ratio: {tracker.metrics['trainable_ratio']*100:.4f}%")

# Compare with LoRA
print("\n--- Comparison ---")
print("Prompt Tuning trains ONLY the soft prompt embeddings")
print("This is typically 10-100x fewer parameters than LoRA!")

In [ ]:
# Visualize trainable parameters
print("\nTrainable layers:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"  {name}: {param.shape}")

## 5. Format Training Data

In [ ]:
def format_instruction(sample):
    """Format instruction-response pair for training."""
    instruction = sample["instruction"]
    response = sample["output"]
    
    # Same format as LoRA for fair comparison
    text = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{response}<|eot_id|>"
    
    return {"text": text}

# Apply formatting
train_dataset = train_dataset.map(format_instruction, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(format_instruction, remove_columns=val_dataset.column_names)

print(f"Formatted train dataset: {train_dataset}")

## 6. Configure Training

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Output directory for Prompt Tuning
OUTPUT_DIR = f"{PROJECT_ROOT}/models/prompt_tuning/checkpoint"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Output directory: {OUTPUT_DIR}")

In [ ]:
# Training arguments (same hyperparameters as LoRA for fair comparison)
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    
    # Training hyperparameters (same as LoRA)
    num_train_epochs=config['training']['sft_epochs'],
    per_device_train_batch_size=config['training']['sft_batch_size'],
    per_device_eval_batch_size=config['training']['sft_batch_size'],
    gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
    
    # Optimizer (slightly higher LR often works better for prompt tuning)
    learning_rate=3e-4,  # Higher than LoRA's 2e-4
    weight_decay=0.01,
    optim="paged_adamw_32bit",
    
    # Learning rate schedule
    lr_scheduler_type="cosine",
    warmup_steps=100,
    
    # Logging
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=10,
    logging_first_step=True,
    
    # Evaluation
    evaluation_strategy="steps",
    eval_steps=200,
    
    # Checkpointing
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    
    # Performance
    fp16=True,
    gradient_checkpointing=True,
    
    # Misc
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

print("Training arguments configured!")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Learning rate: {training_args.learning_rate}")

## 7. Initialize Trainer

In [ ]:
# SFT Trainer with Prompt Tuning
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=prompt_tuning_config,
    dataset_text_field="text",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_args,
    packing=False,
)

print("Trainer initialized!")

## 8. Train Model

In [ ]:
# Start training with timing
print("Starting Prompt Tuning training...")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 50)

tracker.start_training()
train_result = trainer.train()
tracker.end_training()

print("\n" + "=" * 50)
print(f"Training completed!")
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Training time: {tracker.metrics['training_time_seconds']/3600:.2f} hours")

In [ ]:
# Print training metrics
print("\nTraining metrics:")
print(f"  Final train loss: {train_result.training_loss:.4f}")
print(f"  Total steps: {train_result.global_step}")

## 9. Evaluate Model

In [ ]:
# Evaluate on validation set
print("Evaluating model...")
eval_results = trainer.evaluate()

print("\nEvaluation metrics:")
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

## 10. Test Generation & Measure Inference Speed

In [ ]:
def generate_response(instruction: str, max_new_tokens: int = 256):
    """Generate a response for the given instruction."""
    prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        start_time = time.time()
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )
        gen_time = time.time() - start_time
    
    tokens_generated = outputs.shape[1] - inputs['input_ids'].shape[1]
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    # Extract response
    if "<|start_header_id|>assistant<|end_header_id|>" in generated:
        response = generated.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
        response = response.split("<|eot_id|>")[0].strip()
        return response, tokens_generated, gen_time
    
    return generated, tokens_generated, gen_time

print("Generation function defined!")

In [ ]:
# Test generation and measure inference speed
test_instructions = [
    "Explain the concept of machine learning in simple terms.",
    "Write a Python function to calculate the factorial of a number.",
    "What are the main differences between supervised and unsupervised learning?",
]

print("Testing Prompt Tuning model generation:")
print("=" * 50)

total_tokens = 0
total_time = 0

for i, instruction in enumerate(test_instructions):
    print(f"\n[Test {i+1}]")
    print(f"Instruction: {instruction}")
    print(f"\nResponse:")
    response, tokens, gen_time = generate_response(instruction, max_new_tokens=200)
    print(response)
    print(f"\nTokens: {tokens}, Time: {gen_time:.2f}s, Speed: {tokens/gen_time:.1f} tok/s")
    print("-" * 50)
    
    total_tokens += tokens
    total_time += gen_time

# Log inference speed
tracker.log_inference_speed(total_tokens, total_time)
print(f"\nAverage inference speed: {tracker.metrics['inference_tokens_per_sec']:.1f} tokens/sec")

## 11. Save Model

In [ ]:
# Save the final model
FINAL_MODEL_DIR = f"{PROJECT_ROOT}/models/prompt_tuning/final"
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)

print(f"Saving final model to: {FINAL_MODEL_DIR}")

trainer.save_model(FINAL_MODEL_DIR)
tokenizer.save_pretrained(FINAL_MODEL_DIR)

print("Model saved!")

In [ ]:
# Save training configuration
training_config = {
    "method": "prompt_tuning",
    "base_model": BASE_MODEL_ID,
    "training_data_size": len(train_data),
    "validation_data_size": len(val_data),
    "prompt_tuning_config": {
        "num_virtual_tokens": prompt_tuning_config.num_virtual_tokens,
        "init_method": str(prompt_tuning_config.prompt_tuning_init),
    },
    "training_args": {
        "epochs": training_args.num_train_epochs,
        "batch_size": training_args.per_device_train_batch_size,
        "gradient_accumulation_steps": training_args.gradient_accumulation_steps,
        "learning_rate": training_args.learning_rate,
    },
    "results": {
        "train_loss": train_result.training_loss,
        "eval_loss": eval_results["eval_loss"],
        "total_steps": train_result.global_step,
    },
    "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
}

config_path = f"{FINAL_MODEL_DIR}/training_config.json"
with open(config_path, 'w') as f:
    json.dump(training_config, f, indent=2)

print(f"Training config saved to: {config_path}")

## 12. Save Efficiency Metrics

In [ ]:
# Save efficiency metrics for comparison
METRICS_DIR = f"{PROJECT_ROOT}/evaluation/metrics"
os.makedirs(METRICS_DIR, exist_ok=True)

# Get summary
summary = tracker.get_summary()

# Add training results
summary["train_loss"] = train_result.training_loss
summary["eval_loss"] = eval_results["eval_loss"]

# Save
tracker.save(f"{METRICS_DIR}/prompt_tuning_metrics.json")

print("\n=== Efficiency Metrics Summary ===")
for key, value in summary.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

## 13. Comparison Preview

In [ ]:
# Load LoRA metrics if available for comparison
lora_metrics_path = f"{METRICS_DIR}/lora_metrics.json"

if os.path.exists(lora_metrics_path):
    with open(lora_metrics_path, 'r') as f:
        lora_metrics = json.load(f)
    
    print("\n=== Method Comparison ===")
    print(f"{'Metric':<30} {'LoRA':>15} {'Prompt Tuning':>15}")
    print("=" * 60)
    
    comparisons = [
        ("Trainable Params", "trainable_params"),
        ("Trainable Ratio (%)", "trainable_ratio_percent"),
        ("Peak Memory (GB)", "peak_memory_gb"),
        ("Training Time (hours)", "training_time_hours"),
        ("Inference Speed (tok/s)", "inference_tokens_per_sec"),
    ]
    
    for label, key in comparisons:
        lora_val = lora_metrics.get(key, "N/A")
        pt_val = summary.get(key, "N/A")
        
        if isinstance(lora_val, (int, float)) and isinstance(pt_val, (int, float)):
            if key == "trainable_params":
                print(f"{label:<30} {lora_val:>15,} {pt_val:>15,}")
            else:
                print(f"{label:<30} {lora_val:>15.4f} {pt_val:>15.4f}")
        else:
            print(f"{label:<30} {str(lora_val):>15} {str(pt_val):>15}")
else:
    print("LoRA metrics not found. Run 05_sft_training.ipynb with metrics tracking first.")
    print("Full comparison will be available in 09_comparative_analysis.ipynb")

## 14. Cleanup

In [ ]:
# Free GPU memory
del model
del trainer
gc.collect()
torch.cuda.empty_cache()

print("Memory cleared!")

## ✓ Prompt Tuning Complete!

### Summary:
- Prompt Tuning model saved to `models/prompt_tuning/final/`
- Efficiency metrics saved for comparison
- Only soft prompt embeddings were trained (minimal parameters)

### Key Differences from LoRA:
- **Parameters**: ~0.01% vs LoRA's ~0.1-0.5%
- **Training**: Faster due to fewer gradients
- **Memory**: Lower peak usage
- **Performance**: Often slightly lower quality than LoRA

### Next Steps:
1. Run `09_comparative_analysis.ipynb` for full comparison
2. Compare benchmark scores between methods
3. Include results in your report